# Subreddit Prediction 

In [ ]:
subreddit_train = "coursework_subreddit_train.json"
subreddit_test = "coursework_subreddit_test.json"

!gsutil cp gs://textasdata/coursework/coursework_subreddit_train.json $subreddit_train 
!gsutil cp gs://textasdata/coursework/coursework_subreddit_test.json  $subreddit_test


## Data manipulation and analysis 

In [ ]:
import pandas as pd
import json

# A temporary variable to store the list of post content.
posts_tmp = list()

with open(subreddit_train) as jsonfile:
  for i, line in enumerate(jsonfile):
    thread = json.loads(line)
    for post in thread['posts']:
      # Keep the thread title and subreddit with each post.
      posts_tmp.append((thread['title'], post.get("author"), post.get('body')))
print(len(posts_tmp))#
# Create the posts data frame.  
labels = ['title', 'author', 'body']
post_frame = pd.DataFrame(posts_tmp, columns=labels)


In [ ]:
# import gzip

a=[]
b=[]
c=[]
for i,title,author,body in post_frame.itertuples(index=True, name='Pandas'):
  if title in a:
    if author!=None and body!=None:
      b[a.index(title)].append(author)
      c[a.index(title)].append(body)
    if author==None and body!=None:
      b[a.index(title)].append("None")
      c[a.index(title)].append(body)
    if author!=None and body==None:
      b[a.index(title)].append(author)
      c[a.index(title)].append("None")
    if author==None and body==None:
      b[a.index(title)].append("None")
      c[a.index(title)].append("None")
    
  else:    
    
    
    if author==None and body!=None:
      a.append(title)
      c.append([body])
      b.append(["None"])
    
    
    
    if author!=None and body!=None:
      a.append(title)
      c.append([body])
      b.append([author])

    if author!=None and body==None:
      a.append(title)
      c.append(["None"])
      b.append([author])
      
    
author=[]
body=[]
for i in b:
  author.append(" ".join(i))
for i in c:
  body.append(" ".join(i))  
df=pd.DataFrame()
df["title"]=a
df["author"]=author
df["body"]=body

post_frame=df




In [ ]:
posts_test = list()

with open(subreddit_test) as jsonfile:
  for i, line in enumerate(jsonfile):
    thread = json.loads(line)
    for post in thread['posts']:
      # Keep the thread title and subreddit with each post.
      posts_test.append((thread['title'], post.get("author"), post.get('body')))
print(len(posts_test))#
# Create the posts data frame.  
labels = ['title', 'author', 'body']
post_frametest = pd.DataFrame(posts_test, columns=labels)


In [ ]:
post_frame

In [ ]:
a=[]
b=[]
c=[]
for i,title,author,body in post_frametest.itertuples(index=True, name='Pandas'):
  
  
  
  
  if title in a:
    if author!=None and body!=None:
      b[a.index(title)].append(author)
      c[a.index(title)].append(body)
      
    if author==None and body==None:
      b[a.index(title)].append("None")
      c[a.index(title)].append("None")
    
      
    if author==None and body!=None:
      b[a.index(title)].append("None")
      c[a.index(title)].append(body)
    if author!=None and body==None:
      
      b[a.index(title)].append(author)
      c[a.index(title)].append("None")

  else:    
    
    
    if author==None and body!=None:
      a.append(title)
      c.append([body])
      b.append(["None"])
    
    if author!=None and body!=None:
      a.append(title)
      c.append([body])
      b.append([author])

    if author!=None and body==None:
      a.append(title)
      c.append(["None"])
      b.append([author])
      
    
author=[]
body=[]
for i in b:
  author.append(" ".join(i))
for i in c:
  body.append(" ".join(i))  
df1=pd.DataFrame()
df1["title"]=a
df1["author"]=author
df1["body"]=body

post_frametest=df1


In [ ]:
import pandas as pd
#post_frame
#train_data
train_threads = pd.read_json(path_or_buf=subreddit_train, lines=True)
print(list(train_threads.columns.values))
print(train_threads)
#print(train_threads.size)


In [ ]:
test_threads = pd.read_json(path_or_buf=subreddit_test, lines=True)
print(test_threads.head())
print(test_threads.size)



## Text classfication 

In [ ]:
subreddit_counts = train_threads['subreddit'].value_counts()
print(subreddit_counts.describe())
top_subbreddits = subreddit_counts.nlargest(20)
top_subbreddits_list = top_subbreddits.index.tolist()
#print(top_subbreddits)

In [ ]:
train_labels = train_threads['subreddit']
test_labels = test_threads['subreddit']
len(train_labels)

In [ ]:
import spacy

# Load the medium english model. 
# We will use this model to get embedding features for tokens later.
!python -m spacy download en_core_web_md

nlp = spacy.load('en_core_web_md', disable=['ner'])
nlp.remove_pipe('tagger')
nlp.remove_pipe('parser')
# Download a stopword list
import nltk
nltk.download('stopwords')

In [ ]:
#@Tokenize
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#@Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit)):
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

In [ ]:
tokenize_normalize("the app is fun. very happy.")


### Bernoulli Naive Bayes




In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import fbeta_score

def evaluation_summary(description, predictions, true_labels):
  print("Evaluation for: " + description)
  precision = precision_score(predictions, true_labels,average="macro")
  recall = recall_score(predictions, true_labels,average="macro")
  accuracy = accuracy_score(predictions, true_labels)
  f1 = fbeta_score(predictions, true_labels, 1,average="macro") #1 means f_1 measure
  print("Classifier '%s' has Acc=%0.3f P=%0.3f R=%0.3f F1=%0.3f" % (description,accuracy,precision,recall,f1))
  print(classification_report(predictions, true_labels, digits=3))
  print('\nConfusion matrix:\n',confusion_matrix(true_labels, predictions)) # Note the order here is true, predicted, odd.


In [ ]:
  #print(nb_model.predict(train_features))


### Logistic regression


In [ ]:
from sklearn.linear_model import LogisticRegression
#lr = LogisticRegression(solver='saga')
#lr_model = lr.fit(train_features, train_labels)




In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.    """

    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [ ]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

# Use FeatureUnion to combine the features from text and summary
prediction_pipeline = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('text', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('one-hot', CountVectorizer(tokenizer=tokenize_normalize, binary=True)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('one-hot', CountVectorizer(tokenizer=tokenize_normalize, binary=True)), 
              ])),
             ('name', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('one-hot', CountVectorizer(tokenizer=tokenize_normalize, binary=True)), 
              ])),
              
        ])
        )
    ])





In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

prediction_pipelineno = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('text', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
             ('name', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
        ])
        )
    ])


In [ ]:
one_hot_train_features = prediction_pipeline.fit_transform(post_frame)
#one_hot_validation_features = prediction_pipeline.transform(validation_data)
one_hot_test_features = prediction_pipeline.transform(post_frametest)

In [ ]:
one_hot_train_featuresno = prediction_pipelineno.fit_transform(post_frame)
#one_hot_validation_features = prediction_pipeline.transform(validation_data)
one_hot_test_featuresno = prediction_pipelineno.transform(post_frametest)


In [ ]:
lr = LogisticRegression(solver='saga')
combined_model = lr.fit(one_hot_train_features,train_labels)
evaluation_summary("LR TFIDF", lr.predict(one_hot_test_features), test_labels)     ###best classifier 

Confusion matrix 

In [ ]:
# Create Confusion Matrix

import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels


def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes
    
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax


np.set_printoptions(precision=2)


plot_confusion_matrix(test_labels, lr.predict(one_hot_test_features), test_labels, title='Confusion matrix')
plt.show()

In [ ]:
lr = LogisticRegression(solver='saga')
combined_model = lr.fit(one_hot_train_features,train_labels)
#evaluation_summary("LR TFIDF", lr.predict(one_hot_train_features), train_labels)   

Starting TF

In [ ]:
lr = LogisticRegression(solver='saga')
combined_model = lr.fit(one_hot_train_featuresno,train_labels)


In [ ]:
lr = LogisticRegression()
combined_model = lr.fit(one_hot_train_featuresno,train_labels)
evaluation_summary("LR TFIDF", lr.predict(one_hot_test_featuresno), test_labels)     

### SVM

In [ ]:
 from sklearn import svm
X = [[0, 0], [1, 1]]
 y = [0, 1]
 clf = svm.SVC(gamma='scale',kernel='rbf')
 clf.fit(one_hot_train_features,train_labels)  

In [ ]:
evaluation_summary("SVM HOT", clf.predict(one_hot_test_features), test_labels)     

In [ ]:
evaluation_summary("SVM tf", clf.predict(one_hot_test_featuresno), test_labels)     

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_prior = DummyClassifier(strategy='stratified')
dummy_prior.fit(one_hot_train_features, train_labels)
print(dummy_prior.score(one_hot_test_features, test_labels))
evaluation_summary("Dummy Prior", dummy_prior.predict(one_hot_test_features), test_labels)

dummy_mf = DummyClassifier(strategy='most_frequent')
dummy_mf.fit(one_hot_train_features, train_labels)
print(dummy_mf.score(one_hot_test_features, test_labels))
evaluation_summary("Dummy Majority", dummy_mf.predict(one_hot_test_features), test_labels)


In [ ]:
dummy_prior1 = DummyClassifier(strategy='stratified')
dummy_prior1.fit(one_hot_train_featuresno, train_labels)
print(dummy_prior1.score(one_hot_test_featuresno, test_labels))
evaluation_summary("Dummy Prior", dummy_prior1.predict(one_hot_test_featuresno), test_labels)

dummy_mf1 = DummyClassifier(strategy='most_frequent')
dummy_mf1.fit(one_hot_train_featuresno, train_labels)
print(dummy_mf1.score(one_hot_test_featuresno, test_labels))
evaluation_summary("Dummy Majority", dummy_mf1.predict(one_hot_test_featuresno), test_labels)


###Random forest 

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(one_hot_train_features, train_labels)
#evaluation_summary("rf hot", rf.predict(one_hot_train_features), train_labels)


In [ ]:
rf = RandomForestClassifier()
rf.fit(one_hot_train_features, train_labels)
evaluation_summary("rf hot", rf.predict(one_hot_test_features), test_labels)


In [ ]:
rf1 = RandomForestClassifier()

rf1.fit(one_hot_train_featuresno, train_labels)
evaluation_summary("rf tf", rf.predict(one_hot_train_featuresno), train_labels)


In [ ]:

evaluation_summary("rf tf", rf.predict(one_hot_test_featuresno), test_labels)


bar chart 

---



---



In [ ]:

sorted((pd.DataFrame(test_labels).subreddit.unique()))

  queation twoooooooooo

---



---



# Categorical classification with texts

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression



# prediction_pipeline = Pipeline([
#               ('selector', ItemSelector(key='body')),
#               ('one-hot', CountVectorizer(tokenizer=tokenize_normalize)),
#               ('logreg', LogisticRegression(solver='saga'))
#               ])




prediction_pipelineno = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('text', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 
              ])),
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=True,max_features=None,ngram_range=(1,1))), 
              ])),
             ('name', Pipeline([
              ('selector', ItemSelector(key='author')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 
              ])),
              
        ])

        ),              
        ('logreg', LogisticRegression(solver='saga',C= 10, multi_class = "multinomial")),

    
    ])
                  


In [ ]:

prediction_pipelineno.fit(post_frame, train_labels)
evaluation_summary("LR", prediction_pipelineno.predict(post_frametest), test_labels) 


### Error analysis 

In [ ]:
def print_errors(labels, predictions, data):
  label_arr=labels.values
  for idx, prediction in enumerate(predictions):
    label=label_arr[idx]
    if prediction != label:
      print(post_frametest.iloc[idx,1], label, prediction, "                        " ,'label:', label,"              ", 'prediction:', prediction)

In [ ]:

print_errors(test_labels, prediction_pipelineno.predict(post_frametest), post_frametest)

In [ ]:


post_frame1 = post_frame
post_frametest1 =post_frametest
# post_frame1["body_length"]= post_frame1.apply(lambda row: str(len(row.body)), axis=1)
# post_frametest1["body_length"]= post_frametest1.apply(lambda row: str(len(row.body)), axis=1)


# #post_frametest1
# post_frame.drop(["body_length"], axis=1)


VaderSentiment for emotions

In [ ]:
# x = "noob"
# if "noob" in post_frame.iloc[1,2]:
#    print ("success")
# post_frame.iloc[34,2]
#range(len((post_frame)))

!pip install vaderSentiment

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()
sentence = "im feeling good"
vs = analyzer.polarity_scores(post_frame1.iloc[2,2])
vs=analyzer.polarity_scores(sentence)
print(vs)


vaderSentiment

In [ ]:

#!pip install vaderSentiment
#adding emotions 

y = list()
t = list(range(0,len(post_frame1)))
d = list(range(0,len(post_frame1)))

for i in range(len(post_frame1)):
  t[i] = analyzer.polarity_scores(post_frame1.iloc[i,2])
  if t[i]['compound'] >= 0.05:
     d[i] = "positive"
      
  elif t[i]['compound'] <=  -0.05:
     d[i] = "negative"
      
  else: 
    d[i] = "neutral"
    #print('Success!')
  y.append(d[i])

post_frame1["mood"]= y

#okenize_normalize(vs)
# Example code:
# for i in range(len(post_frame1)):
#     vs = analyzer.polarity_scores(post_frame1.iloc[i,2])
#     print("{:-<65} {}".format(post_frame1.iloc[i,2], str(vs)))

In [ ]:
y1 = list()
t1 = list(range(0,len(post_frametest1)))
d1 = list(range(0,len(post_frametest1)))

for i in range(len(post_frametest1)):
  t1[i] = analyzer.polarity_scores(post_frametest1.iloc[i,2])
  if t1[i]['compound'] >= 0.05:
     d1[i] = "positive"
      
  elif t1[i]['compound'] <=  -0.05:
     d1[i] = "negative"
      
  else: 
    d1[i] = "neutral"
    #print('Success!')
  y1.append(d1[i])

post_frametest1["mood"]= y1


token search 

In [ ]:
x = list()
l= list(range(0,len(post_frame1)))

for i in range(len(post_frame1)):
  if 'riot' in post_frame1.iloc[i,2]:
    l[i] = 1
  elif "Riot" in post_frame1.iloc[i,2]:
    l[i] = 1
  elif "LCS" in post_frame1.iloc[i,2]:
    l[i] = 1
  elif "lcs" in post_frame1.iloc[i,2]:
    l[i] = 1
  elif "jungler" in post_frame1.iloc[i,2]:
    l[i] = 1
  elif "laner" in post_frame1.iloc[i,2]:
    l[i] = 1
  else: 
    l[i] = 0
    #print('Success!')
  x.append(str(l[i]))  
  
    
print(len([i for i, e in enumerate(x) if e == str(1)]))
post_frame1["League"]= x
post_frame1 =pd.DataFrame(post_frame)
#post_frame1




In [ ]:
x1 = list()
l1= list(range(0,len(post_frametest1)))

for i in range(len(post_frametest1)):
  if 'riot' in post_frametest1.iloc[i,2]:
    l1[i] = 1
  elif "Riot" in post_frametest1.iloc[i,2]:
    l1[i] = 1
  elif "LCS" in post_frametest1.iloc[i,2]:
    l1[i] = 1
  elif "lcs" in post_frametest1.iloc[i,2]:
    l1[i] = 1
  elif "jungler" in post_frametest1.iloc[i,2]:
    l1[i] = 1
  elif "laner" in post_frame1.iloc[i,2]:
    l[i] = 1
  else:
    l1[i] = 0
    #print('Success!')
  x1.append(str(l1[i]))  
len([i for i, e in enumerate(x1) if e == str(1)])
post_frametest1["League"]= x1
post_frametest1 = pd.DataFrame(post_frametest1)


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


prediction_pipelineno1 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('text', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 
              ])),
              
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=True,max_features=None,ngram_range=(1,1))), 
              ])),
              
            ('name', Pipeline([
             ('selector', ItemSelector(key='author')),
             ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 
             ])),
              
           ('mood', Pipeline([
            ('selector', ItemSelector(key='mood')),
            ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize)), 
            ])),
              
           ('league', Pipeline([
            ('selector', ItemSelector(key='League')),
            ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              
            ]))
              
              
        ])

        ),              
        ('logreg', LogisticRegression(solver='saga',C= 1000,multi_class = "multinomial")),

    
    ])

In [ ]:
train_labels = train_threads['subreddit']
test_labels = test_threads['subreddit']
prediction_pipelineno1.fit(post_frame1, train_labels)
evaluation_summary("LR", prediction_pipelineno1.predict(post_frametest1), test_labels) 


## Discourse prediction ##

In [ ]:
discourse_train = "coursework_discourse_train.json"
discourse_test = "coursework_discourse_test.json"
  
!gsutil cp gs://textasdata/coursework/coursework_discourse_train.json $discourse_train  
!gsutil cp gs://textasdata/coursework/coursework_discourse_test.json  $discourse_test

In [ ]:
# The reddit thread structure is nested with posts in a new content.
# This block reads the file as json and creates a new data frame.
import pandas as pd
import json

def load_posts(file):
  #########################################################################
 # return pd.DataFrame(posts_tmp, columns=labels)
  posts_tmp = list()

  with open(file) as jsonfile:
    for i, line in enumerate(jsonfile):
     # if (i > 2): break
      thread = json.loads(line)
      for post in thread['posts']:
        # NOTE: This could be changed to use additional features from the post or thread.
        # DO NOT change the labels for the test set.
        posts_tmp.append((thread['subreddit'], thread['title'],str(thread['is_self_post']),#str(post.get['is_self_post', ""]),
                     post.get('author', ""), post.get('body', ""), post.get('majority_type', ""),post.get("majority_link", ""),
                          post.get('in_reply_to', ""), (post.get("is_first_post"))
                       ))

# Create the posts data frame.  
  labels = ['subreddit', 'title',"is_self_post", 'author', 'body', 
          "discourse_type","majority_link","in_reply_to","is_first_post"]
  return pd.DataFrame(posts_tmp, columns=labels)



In [ ]:
train_posts = load_posts(discourse_train)
# Filter out empty labels
train_posts = train_posts[train_posts['discourse_type'] != ""]
print(train_posts.head())
print("Num posts: ", train_posts.size)


In [ ]:
(train_posts)

The label for the post we will be predicting is in the discourse_type column.

In [ ]:
test_posts = load_posts(discourse_test)
# Filter out empty labels
test_posts = test_posts[test_posts['discourse_type'] != ""]
print("Num posts: ", test_posts.size)


In [ ]:
train_labels = train_posts['discourse_type']
test_labels = test_posts['discourse_type']


Examine the distribution over labels on the training data.

In [ ]:
discourse_counts = train_labels.value_counts()
print(discourse_counts.describe())

top_discourse = discourse_counts.nlargest(200)
print(top_discourse)
top_discourse = top_discourse.index.tolist()
print(top_discourse)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
labels = ['subreddit', 'title', 'url', 'author', 'body', 'majority_link', 
           'discourse_type', 'in_reply_to']

prediction_pipelinetfq3 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
#             ('subreddit', Pipeline([
#               ('selector', ItemSelector(key='subreddit')),
#               ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,ngram_range=(0,3))), 
#               ])),
              
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=True)), 
              ])),
              
              
            ('name', Pipeline([
             ('selector', ItemSelector(key='author')),
             ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
                           #   ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 

             ])),
              

     ##################extra         
               
              ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
           
              

              ('majority', Pipeline([
              ('selector', ItemSelector(key='majority_link')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('reply', Pipeline([
              ('selector', ItemSelector(key='in_reply_to')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('self', Pipeline([
              ('selector', ItemSelector(key='is_self_post')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
        ])

        ),              
        ('logreg', LogisticRegression(solver='saga',C= 1000,multi_class = "multinomial")),

    
    ])

In [ ]:
prediction_pipelinetfq3.fit(train_posts, train_labels)
evaluation_summary( "LR", prediction_pipelinetfq3.predict(test_posts), test_labels) 


In [ ]:
plot_confusion_matrix(test_labels, prediction_pipelinetfq3.predict(test_posts), test_labels, title='Confusion matrix')
plt.show()

In [ ]:
def print_errors(labels, predictions, data):
  label_arr=labels.values
  for idx, prediction in enumerate(predictions):
    label=label_arr[idx]
    if prediction != label:
      print(test_posts.iloc[idx,4], label, prediction, "                        " ,'label:', label,"              ", 'prediction:', prediction)
print_errors(test_labels, prediction_pipelinetfq3.predict(test_posts), test_labels)

In [ ]:
def spacy_tokenize(string):
  tokens = list()
  doc = nlp(string)
  for token in doc:
    tokens.append(token)
  return tokens

#@Normalize
def normalize(tokens):
  normalized_tokens = list()
  for token in tokens:
    normalized = token.text.lower().strip()
    if ((token.is_alpha or token.is_digit)):
      normalized_tokens.append(normalized)
  return normalized_tokens
  return normalized_tokens

#@Tokenize and normalize
def tokenize_normalize(string):
  return normalize(spacy_tokenize(string))

In [ ]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

class AverageEmbeddingTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.dimension = 300
        
    def fit(self, X, y):
        return self
      
    def transform(self, X):  
      # Skip OOV terms. 
      # Return 0 for all dimensions if no words are in the vocabulary.
      dense_matrix =  np.array([ 
          np.mean([token.vector for token in self.tokenizer(doc) if not token.is_oov]
                or [np.zeros(self.dimension)], axis=0)
          for doc in X
      ])
      return dense_matrix

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
import numpy as np

class baseTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
      return None
        
    def fit(self, X, y):
        return self
      
    def transform(self, X):  
      return X[:,np.newaxis]

In [ ]:
train_posts["body_length"]= train_posts.apply(lambda row: len((row.body)), axis=1)
test_posts["body_length"]= test_posts.apply(lambda row: len((row.body)), axis=1)


# embedding_vectorizer = AverageEmbeddingTransformer(spacy_tokenize)
# train_embedding_features = embedding_vectorizer.transform(train_posts['body_length'])

Transform is first post in to 1 and 0 

In [ ]:
#train_posts["body_length"] = train_embedding_features
# x = list()
# l= list(range(0,len(post_frame1)))

# for i in range(len(post_frame1)):
#   if 'riot' in post_frame1.iloc[i,2]:
#     l[i] = 1
#   elif "Riot" in post_frame1.iloc[i,2]:
#     l[i] = 1
#   elif "LCS" in post_frame1.iloc[i,2]:
#     l[i] = 1
#   elif "lcs" in post_frame1.iloc[i,2]:
#     l[i] = 1
#   elif "jungler" in post_frame1.iloc[i,2]:
#     l[i] = 1
#   elif "laner" in post_frame1.iloc[i,2]:
#     l[i] = 1
#   else: 
#     l[i] = 0
#     #print('Success!')
#   x.append(str(l[i]))  
  
x = list()
l= list(range(0,len(train_posts)))


#   for t in range(len(train_posts)):
    
#     if str(train_posts.iloc[t,8])==True:
#       l[t] = 1
#     else: 
#       l[t] = 0
#       x.append((l[t]))
# x

for i in range(len(train_posts)):
    if 'True' in str(train_posts.iloc[i,8]):
      l[i] = str(1)
    else:
      l[i]=str(0)




train_posts["is_first_post"] =l

In [ ]:
x = list()
l= list(range(0,len(test_posts)))

for i in range(len(test_posts)):
    if 'True' in str(test_posts.iloc[i,8]):
      l[i] = str(1)
    else:
      l[i]=str(0)




test_posts["is_first_post"] =l

Adding follow up question - answer 

In [ ]:
x12 = list()
l12= ["others"]*len(train_posts)


#   for t in range(len(train_posts)):
    
#     if str(train_posts.iloc[t,8])==True:
#       l[t] = 1
#     else: 
#       l[t] = 0
#       x.append((l[t]))
# x

for i in range(len(train_posts)):
    if "1" in str(train_posts.iloc[i,8]):
      l12[i] = "potential_question"
      l12[i+1]="potential_answer"
      l12[i+2]="others"




#l12
train_posts["follow_posts"] =l12
#train_posts

In [ ]:
x2 = list()
l2= ["others"]*len(test_posts)


#   for t in range(len(train_posts)):
    
#     if str(train_posts.iloc[t,8])==True:
#       l[t] = 1
#     else: 
#       l[t] = 0
#       x.append((l[t]))
# x

for i in range(len(test_posts)):
    if "1" in str(test_posts.iloc[i,8]):
      l2[i] = "potential_question"
      l2[i+1]="potential_answer"
      l2[i+2]="others"




#test_posts

test_posts["follow_posts"] =l2


VaderSentiment for double emtion outputs

In [ ]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
y = list()
t = list(range(0,len(train_posts)))
d = list(range(0,len(train_posts)))

for i in range(len(train_posts)):
  t[i] = analyzer.polarity_scores(train_posts.iloc[i,4])
  if t[i]['compound'] >= 0.05:
     d[i] = "positive"
      
  elif t[i]['compound'] <=  -0.05:
     d[i] = "negative"
      
  else: 
    d[i] = "neutral"
    #print('Success!')
  y.append(d[i])

  
train_posts["mood"]= y
# post_frame1


In [ ]:
y = list()
t = list(range(0,len(test_posts)))
d = list(range(0,len(test_posts)))

for i in range(len(test_posts)):
  t[i] = analyzer.polarity_scores(test_posts.iloc[i,4])
  if t[i]['compound'] >= 0.05:
     d[i] = "positive"
      
  elif t[i]['compound'] <=  -0.05:
     d[i] = "negative"
      
  else: 
    d[i] = "neutral"
    #print('Success!')
  y.append(d[i])

  
test_posts["mood"]= y

Link posts or self wrote 

In [ ]:
x = list()
linktrain= list(range(0,len(train_posts)))

for i in range(len(train_posts)):
    if 'www' in (train_posts.iloc[i,4]):
      linktrain[i] = "linkpost"
      
    elif "http:" in (train_posts.iloc[i,4]):
      linktrain[i]= "linkpost"
      
    else:
      linktrain[i]="selfpost"




train_posts["is_link_post"] =linktrain

In [ ]:

linktest= list(range(0,len(test_posts)))

for i in range(len(test_posts)):
    if 'www' in (test_posts.iloc[i,4]):
      linktest[i] = "linkpost"
      
    elif "http:" in (test_posts.iloc[i,4]):
      linktest[i]= "linkpost"
      
    else:
      linktest[i]="selfpost"




test_posts["is_link_post"] =linktest

Symbols 

In [ ]:

symbols= list(range(0,len(test_posts)))


for i in range(len(test_posts)):
  if '?' in test_posts.iloc[i,4]:
    symbols[i] = "question"
  elif "!" in test_posts.iloc[i,4]:
    symbols[i] = "superise"
  elif ":)" in test_posts.iloc[i,4]:
    symbols[i] = "happy"
  else:
    symbols[i] = "others"
#test_posts["League"]= x1
#test_posts = pd.DataFrame(test_posts)
test_posts["symbols_analysis"]= symbols


symbols= list(range(0,len(train_posts)))

for i in range(len(train_posts)):
  if '?' in train_posts.iloc[i,4]:
    symbols[i] = "question"
  elif "!" in train_posts.iloc[i,4]:
    symbols[i] = "superise"
  elif ":)" in train_posts.iloc[i,4]:
    symbols[i] = "happy"
  else:
    symbols[i] = "others"
train_posts["symbols_analysis"]= symbols


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
labels = ['subreddit', 'title', 'url', 'author', 'body', 'majority_link', 
           'discourse_type', 'in_reply_to']

prediction_pipelinetfq4 = Pipeline([
        ('union', FeatureUnion(
          transformer_list=[
            ('subreddit', Pipeline([
              ('selector', ItemSelector(key='subreddit')),
              ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,ngram_range=(0,3))), 
              ])),
              
            ('title', Pipeline([
              ('selector', ItemSelector(key='title')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=True)), 
              ])),
              
              
            ('name', Pipeline([
             ('selector', ItemSelector(key='author')),
             ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
                           #   ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 

             ])),
              
             ('follow', Pipeline([
             ('selector', ItemSelector(key='follow_posts')),
             ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
                           #   ('one-hot', TfidfVectorizer(tokenizer=tokenize_normalize,sublinear_tf=False,max_features=None,ngram_range=(1,1))), 

             ])),
              

     ##################extra         
               
              ('body', Pipeline([
              ('selector', ItemSelector(key='body')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
#               ('body_length', Pipeline([
#               ('selector', ItemSelector(key='body_length')),
#               ('one-hot', baseTransformer()), 
#               ])),
              

              ('majority', Pipeline([
              ('selector', ItemSelector(key='majority_link')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('reply', Pipeline([
              ('selector', ItemSelector(key='in_reply_to')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('self', Pipeline([
              ('selector', ItemSelector(key='is_self_post')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('first', Pipeline([
              ('selector', ItemSelector(key='is_first_post')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('mood', Pipeline([
              ('selector', ItemSelector(key='mood')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('link', Pipeline([
              ('selector', ItemSelector(key='is_link_post')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              ('symbol', Pipeline([
              ('selector', ItemSelector(key='symbols_analysis')),
              ('tf', TfidfVectorizer(tokenizer=tokenize_normalize)), 
              ])),
              
              
        ])

        ),              
        ('logreg', LogisticRegression(solver='saga',C= 1000,multi_class = "multinomial")),

    
    ])

In [ ]:

model = prediction_pipelinetfq4.fit(train_posts, train_labels)
evaluation_summary( "LR", prediction_pipelinetfq4.predict(test_posts), test_labels) 


In [ ]:
# Define the features
X_train = train_posts["discourse_type"]#features from training data
X_test = test_posts["discourse_type"]#features from test data

from sklearn.dummy import DummyClassifier
clf = DummyClassifier(strategy='stratified',random_state=0)
clf.fit(X_train, train_posts['discourse_type'])
predictions = clf.predict(X_test)  
print(classification_report(predictions, test_posts['discourse_type']))